In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-137052",
            subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7',
            resource_group='aml-quickstarts-137052')

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-137052
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-137052


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cluster = ComputeTarget(workspace=ws, name="project-compute")
    print("Compute already exits!")
except:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cluster = ComputeTarget.create(ws, "project-compute", config)
    print("Compute created!")
    
cluster.wait_for_completion()

Compute already exits!


In [3]:
%%writefile conda_dependencies.yml
 dependencies:
 - python=3.6.2
 - scikit-learn
 - pip:
    - azureml-defaults

Overwriting conda_dependencies.yml


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import ScriptRunConfig, Experiment
import os
from azureml.core import Environment

env = Environment.from_conda_specification(name="sklearn-env", file_path="./conda_dependencies.yml")

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.0, 1.0), 
    "max_iter": choice(50, 100, 150, 200, 250)
})

# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.1,
    evaluation_interval=1,
    delay_evaluation=5
)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=".",
    compute_target=cluster,
    entry_script="train.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=4,
    max_total_runs=40
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
best = hyperdrive_run.get_best_run_by_primary_metric()

In [ ]:
best_metric = best.get_metrics()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best = hyperdrive_run.get_best_run_by_primary_metric()
best_metric = best.get_metrics()
param = best.get_details()['runDefinition']
best.register_model("best_model", "outputs/project_model.joblib")

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
automl_data = TabularDatasetFactory.from_delimited_files(
    path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(automl_data)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.pipeline.steps import AutoMLStep

auto_exp = Experiment(workspace=ws, name="auto_exp")
auto_run = auto_exp.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_auto_run, best_auto_model = auto_run.get_output()
best_auto_run.register_model(model_name="best_auto_model", model_path="./outputs")

In [ ]:
cluster.delete()